# Introduction

Llama 3.1 is currently (as of Aug 2024) the latest model from Meta. Let's try and see what this model is bringing.

Will test now the following model:

* **Model**: Llama3.1
* **Version**: 8b-instruct
* **Framework**: Transformers
* **Version**: V1

This is what we will test:

* Simple prompts with general information questions
* Poetry (haiku, sonets) writing
* Code writing (Python, C++, Java)
* Software design (simple problems)
* Multi-parameter questions
* Chain of reasoning
* A more complex reasoning problem


**Note**: this notebook is organized to facilitate a comparison of **Llama3.1** with **Gemma 2**. Corresponding notebook is [Unlock the Power of Gemma 2: Prompt it like a Pro](https://www.kaggle.com/code/gpreda/unlock-the-power-of-gemma-2-prompt-it-like-a-pro). At the end of current notebook we are reviewing the performance of the two notebooks for the list of tasks submitted to both.

# Preparation

We import the libraries we need, and we set the model to be ready for testing.  

We will use `transformers` library from HuggingFace. We will also need to install `accelerate`.

## Import packages

In [1]:
%%capture
%pip install -U transformers accelerate

In [2]:
from time import time
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
from IPython.display import display, Markdown

2024-08-17 16:15:25.623986: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-17 16:15:25.624140: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-17 16:15:25.763439: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Define model

This step might take a bit more time, around 2 minutes.

In [3]:
model_id = "/kaggle/input/llama-3.1/transformers/8b-instruct/1"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
        model_id,
        return_dict=True,
        low_cpu_mem_usage=True,
        torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [4]:
llama31_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
)

## Prepare query function

In [5]:
def query_model(
        system_message,
        user_message,
        temperature=0.7,
        max_length=1024
        ):
    start_time = time()
    user_message = "Question: " + user_message + " Answer:"
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message},
        ]
    prompt = llama31_pipeline.tokenizer.apply_chat_template(
        messages, 
        tokenize=False, 
        add_generation_prompt=True
        )
    terminators = [
        llama31_pipeline.tokenizer.eos_token_id,
        llama31_pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]
    sequences = llama31_pipeline(
        prompt,
        do_sample=True,
        top_p=0.9,
        temperature=temperature,
        num_return_sequences=1,
        eos_token_id=terminators,
        max_new_tokens=max_length,
        return_full_text=False,
        pad_token_id=terminators[0]
    )
    #answer = f"{sequences[0]['generated_text'][len(prompt):]}\n"
    answer = sequences[0]['generated_text']
    end_time = time()
    ttime = f"Total time: {round(end_time-start_time, 2)} sec."

    return user_message + " " + answer  + " " +  ttime


system_message = """
You are an AI assistant designed to answer simple questions.
Please restrict your answer to the exact question asked.
"""

## Utility function for formatting the output


In [6]:
def colorize_text(text):
    for word, color in zip(["Reasoning", "Question", "Answer", "Total time"], ["blue", "red", "green", "magenta"]):
        text = text.replace(f"{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

# Test the model


## Test with few simple geography and history questions

In [7]:
t1 = time()
response = query_model(
    system_message,
    user_message="What is the surface temperature of the Moon?",
    temperature=0.1,
    max_length=256)
display(Markdown(colorize_text(f"{response}")))



**<font color='red'>Question:</font>** What is the surface temperature of the Moon? 

**<font color='green'>Answer:</font>** The surface temperature of the Moon varies greatly between day and night. During the lunar day, which lasts about 29.5 Earth days, the surface temperature can reach as high as 127°C (261°F) in the direct sunlight. In contrast, during the lunar night, the temperature can drop to as low as -173°C (-279°F) in the permanently shadowed craters near the lunar poles. 

**<font color='magenta'>Total time:</font>** 52.7 sec.

Let's ask a more simple question, from European geography.

In [8]:
response = query_model(    
    system_message,
    user_message="What is the surface of France?",
    temperature=0.1,
    max_length=128)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** What is the surface of France? 

**<font color='green'>Answer:</font>** The surface of France is approximately 643,801 square kilometers. 

**<font color='magenta'>Total time:</font>** 8.75 sec.

Now, let's continue with some realy tough questions, from Ancient Greece, medieval Japan and more.

In [9]:
response = query_model(
    system_message,
    user_message="When was the 30 years war?",
    temperature=0.1, 
    max_length=128)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** When was the 30 years war? 

**<font color='green'>Answer:</font>** The 30 Years War was a complex conflict that lasted from 1618 to 1648. 

**<font color='magenta'>Total time:</font>** 13.09 sec.

In [10]:
response = query_model(
    system_message,
    user_message="What is graphe paranomon?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** What is graphe paranomon? 

**<font color='green'>Answer:</font>** Graphe paranomon is a term used in ancient Greek law. It refers to a type of impeachment in ancient Athens, where a citizen could bring a charge against a magistrate for proposing a law that was unconstitutional or contrary to the laws of the city-state. 

**<font color='magenta'>Total time:</font>** 32.79 sec.

In [11]:
response = query_model(
    system_message,
    user_message="Who was the next shogun after Tokugawa Ieyasu?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Who was the next shogun after Tokugawa Ieyasu? 

**<font color='green'>Answer:</font>** Tokugawa Hidetada. 

**<font color='magenta'>Total time:</font>** 5.68 sec.

In [12]:
response = query_model(
    system_message,
    user_message="What was the name of the Chinese dinasty during 1st century B.C.?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** What was the name of the Chinese dinasty during 1st century B.C.? 

**<font color='green'>Answer:</font>** The Han Dynasty. 

**<font color='magenta'>Total time:</font>** 3.19 sec.

Let's try now some questions from American history. These must be simpler.

In [13]:
response = query_model(
    system_message,
    user_message="Who was the first American president?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Who was the first American president? 

**<font color='green'>Answer:</font>** George Washington. 

**<font color='magenta'>Total time:</font>** 2.59 sec.

In [14]:
response = query_model(
    system_message,
    user_message="When took place the Civil War in United States of America?",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** When took place the Civil War in United States of America? 

**<font color='green'>Answer:</font>** The American Civil War took place from 1861 to 1865. 

**<font color='magenta'>Total time:</font>** 10.03 sec.

## Let's write poetry now

In [15]:
system_message = """
You are an AI assistant designed to write poetry.
Please answer with a haiku format (17 words poems).
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a poem about Boris Becker wins in tennis",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a poem about Boris Becker wins in tennis 

**<font color='green'>Answer:</font>** Racket's mighty swing falls
Golden ace on Wimbledon's grass
Becker's triumphant cry 

**<font color='magenta'>Total time:</font>** 13.14 sec.

In [16]:
response = query_model(
    system_message,
    user_message="Please write a poem about Shakespeare being lame at playing poker",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a poem about Shakespeare being lame at playing poker 

**<font color='green'>Answer:</font>** Bard's bluffing fails miserably
Shakespeare's poker face a joke
Lame, his royal flush 

**<font color='magenta'>Total time:</font>** 14.89 sec.

In [17]:
system_message = """
You are an AI assistant designed to write poetry.
Please answer with a short poem, with rime, in the style of Shakespeare's poems.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a poem about Nadia Comaneci winning Montreal Olympiad",
    temperature=0.1, 
    max_length=512)
display(Markdown(colorize_text(response)))

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset




**<font color='red'>Question:</font>** Please write a poem about Nadia Comaneci winning Montreal Olympiad 

**<font color='green'>Answer:</font>** O, fairest Nadia, with thy skills so fine,
Didst conquer Montreal's Olympic shrine.
Thy gymnastics prowess, a wondrous sight,
Didst earn thee perfect tens, in morning light.

Thy routines, a symphony of motion true,
Didst dance upon the floor, with steps anew.
The judges' scores, a chorus of delight,
Didst sing thy praises, through the Montreal night.

Thy name, a byword for excellence and might,
Didst echo through the halls, of Olympic light.
The crowd, a sea of faces, aglow,
Didst cheer thy triumph, as the scores didst flow.

Thou wert the first, to score a perfect score,
A feat most rare, and one to adore.
Thy legacy, a shining star, that doth abide,
A testament to thy skill, and thy gymnast's pride. 

**<font color='magenta'>Total time:</font>** 113.08 sec.

## Math problems and Python code

Let's continue now with some math problems for which the response should be given by Python code.

In [18]:
system_message = """
You are an AI assistant designed to write simple Python code.
Please answer with the listing of the Python code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in Python to calculate the area of a circle of radius r",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in Python to calculate the area of a circle of radius r 

**<font color='green'>Answer:</font>** ```python
import math

def calculate_circle_area(r):
    """
    Calculate the area of a circle given its radius.

    Args:
        r (float): The radius of the circle.

    Returns:
        float: The area of the circle.
    """
    return math.pi * (r ** 2)

# Example usage:
radius = 5
area = calculate_circle_area(radius)
print(f"The area of the circle with radius {radius} is {area:.2f}")
``` 

**<font color='magenta'>Total time:</font>** 62.48 sec.

In [19]:
response = query_model(
    system_message,
    user_message="Please write a function in Python to order a list",
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))




**<font color='red'>Question:</font>** Please write a function in Python to order a list 

**<font color='green'>Answer:</font>** ```python
def order_list(input_list):
    """
    This function takes a list as input and returns the list in sorted order.
    
    Args:
        input_list (list): The list to be sorted.
    
    Returns:
        list: The sorted list.
    """
    return sorted(input_list)

# Example usage:
numbers = [64, 34, 25, 12, 22, 11, 90]
print(order_list(numbers))  # Output: [11, 12, 22, 25, 34, 64, 90]
``` 

**<font color='magenta'>Total time:</font>** 74.62 sec.

## Software design

In [20]:
response = query_model(
    system_message,
    user_message="""Please write a class in Python 
                        to model a phone book (storing name, surname, address, phone) 
                        with add, delete, order by name, search operations.
                        The class should store a list of contacts, each
                        with name, surname, address, phone information stored.
                        """,
    temperature=0.1, 
    max_length=1024)
display(Markdown(colorize_text(response)))




**<font color='red'>Question:</font>** Please write a class in Python 
                        to model a phone book (storing name, surname, address, phone) 
                        with add, delete, order by name, search operations.
                        The class should store a list of contacts, each
                        with name, surname, address, phone information stored.
                         

**<font color='green'>Answer:</font>** ```python
class Contact:
    def __init__(self, name, surname, address, phone):
        self.name = name
        self.surname = surname
        self.address = address
        self.phone = phone

class PhoneBook:
    def __init__(self):
        self.contacts = []

    def add_contact(self, name, surname, address, phone):
        new_contact = Contact(name, surname, address, phone)
        self.contacts.append(new_contact)

    def delete_contact(self, name, surname):
        for contact in self.contacts:
            if contact.name == name and contact.surname == surname:
                self.contacts.remove(contact)
                print(f"Contact {name} {surname} deleted successfully.")
                return
        print(f"Contact {name} {surname} not found.")

    def order_by_name(self):
        self.contacts.sort(key=lambda x: (x.name, x.surname))

    def search_contact(self, name, surname):
        for contact in self.contacts:
            if contact.name == name and contact.surname == surname:
                return contact
        return None

    def print_contacts(self):
        for contact in self.contacts:
            print(f"Name: {contact.name} {contact.surname}, Address: {contact.address}, Phone: {contact.phone}")

# Example usage:
phone_book = PhoneBook()
phone_book.add_contact("John", "Doe", "123 Main St", "123-456-7890")
phone_book.add_contact("Jane", "Doe", "456 Elm St", "987-654-3210")
phone_book.add_contact("Alice", "Smith", "789 Oak St", "555-555-5555")

phone_book.order_by_name()
phone_book.print_contacts()

found_contact = phone_book.search_contact("John", "Doe")
if found_contact:
    print(f"Found contact: {found_contact.name} {found_contact.surname}, Address: {found_contact.address}, Phone: {found_contact.phone}")
else:
    print("Contact not found.")

phone_book.delete_contact("Jane", "Doe")
phone_book.print_contacts()
``` 

**<font color='magenta'>Total time:</font>** 263.96 sec.

In [21]:
response = query_model(
    system_message,
    user_message="""Please write a small Python module that creates a REST API service
                        with two endpoints: 
                        * get_status (GET)
                        * prediction (POST)
                        The prediction endpoint receives the payload, extract three fields: city, street and number
                        and returns a field called price_estimate
                        
                        """,
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a small Python module that creates a REST API service
                        with two endpoints: 
                        * get_status (GET)
                        * prediction (POST)
                        The prediction endpoint receives the payload, extract three fields: city, street and number
                        and returns a field called price_estimate
                        
                         

**<font color='green'>Answer:</font>** ```python
from flask import Flask, request, jsonify

app = Flask(__name__)

# Sample data for demonstration purposes
data = {
    "New York": {"street": "Main St", "number": 123, "price_estimate": 1000},
    "Los Angeles": {"street": "Sunset Blvd", "number": 456, "price_estimate": 800},
    "Chicago": {"street": "Michigan Ave", "number": 789, "price_estimate": 1200}
}

@app.route('/status', methods=['GET'])
def get_status():
    """Return a simple status message"""
    return jsonify({"status": "OK"})

@app.route('/prediction', methods=['POST'])
def prediction():
    """Make a prediction based on the provided data"""
    try:
        city = request.json['city']
        street = request.json['street']
        number = request.json['number']
        
        if city in data and data[city]['street'] == street and data[city]['number'] == number:
            return jsonify({"price_estimate": data[city]['price_estimate']})
        else:
            return jsonify({"error": "City, street or number not found"}), 404
    except KeyError:
        return jsonify 

**<font color='magenta'>Total time:</font>** 158.26 sec.

## C++ code

We continue now with some C++ code.

In [22]:
system_message = """
You are an AI assistant designed to write simple C++ code.
Please answer with the listing of the C++ code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in C++ to calculate the area of a circle of radius r", 
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to calculate the area of a circle of radius r 

**<font color='green'>Answer:</font>** ```cpp
#include <iostream>
#include <cmath>

double calculate_circle_area(double r) {
    return M_PI * pow(r, 2);
}

int main() {
    double r = 5.0;
    std::cout << "The area of the circle is: " << calculate_circle_area(r) << std::endl;
    return 0;
}
``` 

**<font color='magenta'>Total time:</font>** 47.46 sec.

In [23]:
system_message = """
You are an AI assistant designed to write simple C++ code.
Please answer with the listing of the C++ code.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="Please write a function in C++ to calculate the volume of a cylinder with radius r and height h",
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to calculate the volume of a cylinder with radius r and height h 

**<font color='green'>Answer:</font>** ```cpp
#include <iostream>

double cylinder_volume(double r, double h) {
    return 3.14159 * r * r * h;
}

int main() {
    double r = 5.0;
    double h = 10.0;
    std::cout << "The volume of the cylinder is: " << cylinder_volume(r, h) << std::endl;
    return 0;
}
``` 

**<font color='magenta'>Total time:</font>** 54.22 sec.

In [24]:
response = query_model(
    system_message,
    user_message="Please write a function in C++ to order a list", 
    temperature=0.1, 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** Please write a function in C++ to order a list 

**<font color='green'>Answer:</font>** ```cpp
#include <iostream>
#include <vector>
#include <algorithm>

std::vector<int> orderList(std::vector<int> lst) {
    std::sort(lst.begin(), lst.end());
    return lst;
}

int main() {
    std::vector<int> numbers = {64, 34, 25, 12, 22, 11, 90};
    std::vector<int> orderedNumbers = orderList(numbers);
    for (int num : orderedNumbers) {
        std::cout << num << " ";
    }
    return 0;
}
``` 

**<font color='magenta'>Total time:</font>** 73.93 sec.

## Multiple parameters questions

### Best food in France

In [25]:
system_message = """
You are an AI assistant designed to answer questions with parameters.
Return the answer formated nicely, for example with bullet points.
"""
user_message = """
What are the {adjective} {number} {items} from {place}?
"""
response = query_model(
    system_message,
    user_message.format(
    adjective="best",
    number="3",
    items="food",
    place="France"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the best 3 food from France?
 

**<font color='green'>Answer:</font>** Bon appétit! Here are the top 3 iconic French foods that you should try:

* **Escargots (Snails in Garlic Butter)**: A classic French appetizer, typically served as an entrée, escargots are snails cooked in garlic butter, parsley, and white wine. They're a delicacy that's both rich and flavorful.
* **Coq au Vin (Chicken Braised in Red Wine)**: A classic French dish from the Burgundy region, Coq au Vin is a braised chicken cooked in red wine, mushrooms, onions, and bacon. The result is a tender, flavorful chicken dish that's fall-off-the-bone delicious.
* **Croissants**: Flaky, buttery, and crispy, croissants are a quintessential French pastry. Whether plain or filled with chocolate, jam, or ham and cheese, croissants are a delicious breakfast or snack option that's perfect for any time of day.

These three dishes showcase the rich flavors and techniques that make French cuisine so renowned worldwide. Bon appétit! 

**<font color='magenta'>Total time:</font>** 134.97 sec.

### Best attractions in Italy

In [26]:
response = query_model(    
    system_message,
    user_message.format(
    adjective="best",
    number="five",
    items="attractions",
    place="Italy"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the best five attractions from Italy?
 

**<font color='green'>Answer:</font>** Italy, a country known for its rich history, art, architecture, fashion, and delicious food! Here are the top 5 attractions to visit in Italy:

* **The Colosseum (Rome)**: This ancient amphitheater is one of Rome's most iconic landmarks and a must-visit attraction. Learn about the history of gladiators and the Roman Empire while exploring the ancient structure.
* **The Leaning Tower of Pisa (Pisa)**: This famous tower is a marvel of engineering and a popular spot for photos. Take a stroll through the Piazza dei Miracoli and admire the beautiful architecture of the surrounding buildings.
* **The Grand Canal (Venice)**: A romantic gondola ride through the canals of Venice is a quintessential Italian experience. Admire the stunning architecture, bridges, and ornate bridges of this unique city.
* **The Uffizi Gallery (Florence)**: This world-renowned museum is home to some of the most famous paintings in the world, including Botticelli's "The Birth of Venus" and Leonardo da Vinci's "Annunciation."
* **The Amalfi Coast (Amalfi Coast)**: This stunning coastline is a paradise for beach lovers and nature enthusiasts. 

**<font color='magenta'>Total time:</font>** 157.61 sec.

### Most affordable places to stay in Spain 

In [27]:
response = query_model(
    system_message,
    user_message.format(
    adjective="most affordable",
    number="two",
    items="places to retire",
    place="Spain"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the most affordable two places to retire from Spain?
 

**<font color='green'>Answer:</font>** Here are two affordable places to retire in Spain:

**Top 2 Affordable Places to Retire in Spain:**

* **1. Málaga, Andalusia**
	+ Average cost of living index: 73.25 (compared to 100 in Madrid)
	+ Rent: €400-€600 per month for a one-bedroom apartment
	+ Food: €3-€5 per meal at a local restaurant
	+ Healthcare: Access to public healthcare and some private options
	+ Weather: Mild Mediterranean climate, with warm winters and hot summers
	+ Culture: Rich cultural heritage, beautiful beaches, and a vibrant expat community
* **2. Gran Canaria, Canary Islands**
	+ Average cost of living index: 69.25 (compared to 100 in Madrid)
	+ Rent: €300-€500 per month for a one-bedroom apartment
	+ Food: €2-€4 per meal at a local restaurant
	+ Healthcare: Access to public healthcare and some private options
	+ Weather: Subtropical climate, with mild winters and warm summers
	+ Culture: Lush landscapes, beautiful beaches, and a growing expat community

Note: Prices may vary depending on the specific 

**<font color='magenta'>Total time:</font>** 157.97 sec.

### Less known but great places to stay in Romania

In [28]:
response = query_model(    
    system_message,
    user_message.format(
    adjective="Less known but great",
    number="4",
    items="places to stay",
    place="Romania"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the Less known but great 4 places to stay from Romania?
 

**<font color='green'>Answer:</font>** Here are some lesser-known but great places to stay in Romania:

• **Sighisoara Medieval Citadel**: A UNESCO World Heritage Site and one of the best-preserved medieval towns in Europe, Sighisoara is a hidden gem in the heart of Transylvania. Its colorful buildings, cobblestone streets, and stunning architecture make it a unique destination for history buffs and photographers.

• **Maramures County**: Located in the north of Romania, Maramures is a region known for its traditional wooden churches, vibrant culture, and stunning natural beauty. Stay in the town of Sighetu Marmației, which offers a glimpse into traditional Romanian life, or in the village of Breb, which is famous for its wooden churches and traditional festivals.

• **Bran Castle**: While many people visit the nearby Bran Castle, which is often mistakenly referred to as Dracula's Castle, few know about the nearby town of Bran, which offers a more authentic and peaceful experience. Stay in one of the local guesthouses and explore the surrounding forests, mountains, and traditional villages.

• **Oradea**: Located in the northwest of Romania, Oradea is a charming city with a rich history, stunning architecture, and a vibrant cultural scene. Its baroque buildings 

**<font color='magenta'>Total time:</font>** 157.87 sec.

### Best commedies by Shakespeare


In [29]:
response = query_model(
    system_message,
    user_message.format(
    adjective="best",
    number="3",
    items="commedies",
    place="Shakespeare entire creation"
    ), 
    max_length=256)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the best 3 commedies from Shakespeare entire creation?
 

**<font color='green'>Answer:</font>** A question that sparks debate among Shakespeare enthusiasts! While opinions may vary, here are three of the most popular comedies from Shakespeare's works, widely regarded as some of the greatest comedies in the English language:

* **A Midsummer Night's Dream**:
	+ A lighthearted and enchanting play that weaves together love, magic, and mischief in a forest inhabited by fairies, humans, and wild creatures.
	+ Features a star-crossed lovers' story, comedic misunderstandings, and a bumbling weaver named Bottom.
* **Twelfth Night**:
	+ A witty and charming comedy that explores themes of love, identity, and deception in a shipwrecked world.
	+ Follows the story of Viola, a young woman who disguises herself as a man to serve the lovesick Orsino, leading to a series of hilarious misunderstandings and romantic entanglements.
* **As You Like It**:
	+ A pastoral comedy that celebrates the power of love, friendship, and redemption in a forest setting.
	+ Features a strong-willed heroine, Rosalind, who disguises herself as a man to escape her unrequited love and find her own happiness in the company of her banished uncle 

**<font color='magenta'>Total time:</font>** 157.57 sec.

### Most important battles fromn WW2

In [30]:
response = query_model(
    system_message,
    user_message.format(
    adjective="most important",
    number="5",
    items="battles",
    place="WW2"
    ), 
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the most important 5 battles from WW2?
 

**<font color='green'>Answer:</font>** Here are the top 5 most important battles of World War II:

**The Top 5 Most Important Battles of World War II:**

1. **Battle of Stalingrad (August 1942 - February 1943)
	* A decisive turning point on the Eastern Front, where Soviet forces defeated the German Sixth Army, marking a major shift in the war's momentum.
	* Resulted in the loss of over 300,000 German soldiers and a significant weakening of the German army.
2. **Battle of Midway (June 1942)
	* A naval battle in the Pacific, where the United States Navy defeated a Japanese fleet, preventing a potential invasion of Hawaii and the West Coast of the United States.
	* Considered a turning point in the war in the Pacific, as it halted Japanese expansion and set the stage for the Allied counterattack.
3. **Battle of Normandy (D-Day) (June 6, 1944)
	* The largest seaborne invasion in history, where Allied forces landed on the beaches of Normandy, France, and began the liberation of Western Europe from German occupation.
	* A crucial turning point in the war in Europe, as it paved the way for the Allied invasion of the continent.
4. **Battle of El Alamein (October - November 1942)
	* A decisive battle in North Africa, where British forces led by General Bernard Montgomery defeated the German and Italian armies, marking a major shift in the desert war.
	* Resulted in the surrender of Axis forces in North Africa and the eventual Allied invasion of Italy.
5. **Battle of Moscow (October 1941 - January 1942)
	* A prolonged battle in which Soviet forces defended Moscow against a German invasion, marking a significant turning point on the Eastern Front.
	* The Soviet victory at Moscow halted the German advance and set the stage for the Soviet counterattack that would eventually push the Germans back.

These five battles were crucial in shaping the outcome of World War II, with each one having a significant impact on the war's trajectory and the ultimate defeat of the Axis powers. 

**<font color='magenta'>Total time:</font>** 268.72 sec.

## Multiple steps reasoning (task chain)

In [31]:
response = query_model(
    system_message,
    user_message.format(
    adjective="most important",
    number="5",
    items="battles",
    place="WW2"
    ), 
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** 
What are the most important 5 battles from WW2?
 

**<font color='green'>Answer:</font>** Here are the top 5 most important battles of World War 2, in no particular order, along with their significance and outcomes:

• **Battle of Stalingrad (August 1942 - February 1943)**
	+ Location: Stalingrad, Russia
	+ Significance: Marked a major turning point on the Eastern Front, as the Soviet Red Army defeated the German Sixth Army, marking a significant loss for the Axis powers and a crucial victory for the Allies.
	+ Outcome: The Soviet Union gained control of the city, and the German army suffered heavy losses, setting the stage for the eventual defeat of Nazi Germany.

• **Battle of Midway (June 4-7, 1942)**
	+ Location: Midway Atoll, Pacific Ocean
	+ Significance: Considered a turning point in the Pacific Theater, as the United States Navy defeated a Japanese naval force, preventing a potential invasion of Hawaii and halting Japanese expansion in the Pacific.
	+ Outcome: The Japanese lost four aircraft carriers, while the United States lost one, giving the Allies a crucial advantage in the Pacific.

• **Battle of El Alamein (October 23 - November 5, 1942)**
	+ Location: El Alamein, Egypt
	+ Significance: Marked a decisive victory for the Allies in North Africa, as British and Commonwealth forces defeated the German and Italian armies, paving the way for the eventual defeat of the Axis powers in Africa.
	+ Outcome: The Axis powers were forced to retreat, and the Allies gained control of North Africa, setting the stage for the invasion of Italy.

• **Battle of Moscow (October 1941 - January 1942)**
	+ Location: Moscow, Russia
	+ Significance: A crucial battle in the Eastern Front, as the Soviet Red Army defeated the German army's attempt to capture Moscow, preventing a potential collapse of the Soviet government.
	+ Outcome: The Soviet Union gained control of the city, and the German army suffered heavy losses, setting the stage for the eventual defeat of Nazi Germany.

• **Battle of Normandy (June 6, 1944)**
	+ Location: Normandy, France
	+ Significance: Marked the beginning of the end of Nazi Germany's control over Western Europe, as Allied forces landed on the beaches of Normandy and began the liberation of Western Europe.
	+ Outcome: The Allies established a foothold on the continent, and the German army 

**<font color='magenta'>Total time:</font>** 316.56 sec.

## Reasoning like Einstein will do

In [32]:
system_message = """
You are a math professor, smart but cool.
Background: A train traveling from Bucharest to Ploiesti (60 km distance) has the speed of 60 km/h. 
The train starts in Bucharest and travels until Ploiesti, once, only in this direction.
A swallow, flying with 90 km/h, fly from Ploiesti to the moving train.
When it reaches the train, the swallow flies back toward Ploiesti,
ahead of the train. At Ploiesti turns again back and continues to fly back and forth 
(between the train approaching Ploiesti and Ploiesti) until the train reaches Ploiesti. 
The swallow will fly continously all the time the train is traveling from Bucharest to Ploiesti.
Reasoning: Think step by step. Explain your reasoning.
Question: {question}
Answer:
"""
response = query_model(
    system_message,
    user_message="How many kilometers will travel totally the swallow?", 
    temperature=0.1,
    max_length=512)
display(Markdown(colorize_text(response)))



**<font color='red'>Question:</font>** How many kilometers will travel totally the swallow? 

**<font color='green'>Answer:</font>** What a classic problem!

Let's break it down step by step:

1. The train travels from Bucharest to Ploiesti at a speed of 60 km/h, and the distance between the two cities is 60 km. To find the time it takes for the train to reach Ploiesti, we can use the formula:

Time = Distance / Speed
Time = 60 km / 60 km/h = 1 hour

So, the train will reach Ploiesti in 1 hour.

2. The swallow flies from Ploiesti to the train at a speed of 90 km/h. Since the train is moving at 60 km/h, the relative speed of the swallow with respect to the train is 90 km/h - 60 km/h = 30 km/h.

3. The swallow flies back and forth between the train and Ploiesti, so it will cover the distance between the train and Ploiesti twice in each round trip. The distance between the train and Ploiesti is decreasing at a rate of 60 km/h (the speed of the train), so the distance between the train and Ploiesti is decreasing by 60 km every hour.

4. Since the train takes 1 hour to reach Ploiesti, the distance between the train and Ploiesti is decreasing by 60 km in 1 hour. The swallow flies at a speed of 30 km/h relative to the train, so it will cover this distance in:

Time = Distance / Speed
Time = 60 km / 30 km/h = 2 hours

However, this is the time it takes for the swallow to cover the distance between the train and Ploiesti once. Since the train is moving, the distance between the train and Ploiesti is decreasing, so the swallow will need to make multiple trips to cover the entire distance.

5. To find the total distance traveled by the swallow, we need to find the total time it takes for the swallow to cover the entire distance between the train and Ploiesti. Since the train takes 1 hour to reach Ploiesti, and the swallow takes 2 hours to cover the distance between the train and Ploiesti once, the swallow will make 1/2 trips in 1 hour (since it takes 2 hours to cover the distance once).

6. Since the train takes 1 hour to reach Ploiesti, the swallow will make 1 

**<font color='magenta'>Total time:</font>** 317.79 sec.